In [1]:

# Large amount of credit goes to:
# https://github.com/keras-team/keras-contrib/blob/master/examples/improved_wgan.py
# which I've used as a reference for this implementation

from __future__ import print_function, division

from keras.datasets import mnist,fashion_mnist
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, z_disc],
                            outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(100,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # Sample generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    # Train the critic
                    d_loss = self.critic_model.train_on_batch([imgs, noise],
                                                              [valid, fake, dummy])

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.generator_model.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch%d step%d [D loss: %f] [G loss: %f]" % (epoch, global_step, d_loss[0], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch, global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        if not os.path.isdir('images_wpgan_fashion_mnist'):
            os.mkdir('images_wpgan_fashion_mnist')
        fig.savefig("images_wpgan_fashion_mnist/epoch_%d_step_%d.png" % (epoch, global_step))
        plt.close()

if __name__ == '__main__':
    wgan = WGANGP()
    wgan.train(epochs=40, batch_size=32, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 128)       0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch0 step5 [D loss: 22.943289] [G loss: -0.339831]
epoch0 step10 [D loss: 21.458675] [G loss: -0.144946]
epoch0 step15 [D loss: 17.273392] [G loss: -0.525018]
epoch0 step20 [D loss: 13.573129] [G loss: -0.115364]
epoch0 step25 [D loss: 14.534114] [G loss: -0.316096]
epoch0 step30 [D loss: 10.039880] [G loss: -0.077815]
epoch0 step35 [D loss: 12.569810] [G loss: 0.054625]
epoch0 step40 [D loss: 6.742446] [G loss: -0.034419]
epoch0 step45 [D loss: 6.245056] [G loss: -0.126629]
epoch0 step50 [D loss: 4.821493] [G loss: -0.212746]
epoch0 step55 [D loss: 4.030509] [G loss: -0.192218]
epoch0 step60 [D loss: 2.686846] [G loss: -0.232919]
epoch0 step65 [D loss: 4.144965] [G loss: -0.026109]
epoch0 step70 [D loss: 2.731128] [G loss: -0.103495]
epoch0 step75 [D loss: 1.304000] [G loss: -0.289076]
epoch0 step80 [D loss: 0.119938] [G loss: 0.162944]
epoch0 step85 [D loss: 0.632614] [G loss: 0.234114]
epoch0 step90 [D loss: 2.002125] [G loss: 0.154911]
epoch0 step95 [D loss: 0.215398] [G loss: 0.

epoch0 step765 [D loss: -1.213223] [G loss: -0.909527]
epoch0 step770 [D loss: -1.226186] [G loss: -0.878493]
epoch0 step775 [D loss: -1.310730] [G loss: -0.842524]
epoch0 step780 [D loss: -0.747169] [G loss: -0.887732]
epoch0 step785 [D loss: -0.641221] [G loss: -0.764324]
epoch0 step790 [D loss: -0.587612] [G loss: -0.961845]
epoch0 step795 [D loss: -0.163173] [G loss: -0.660004]
epoch0 step800 [D loss: -0.246516] [G loss: -0.793672]
epoch0 step805 [D loss: -0.279666] [G loss: -0.695407]
epoch0 step810 [D loss: -1.112405] [G loss: -0.358812]
epoch0 step815 [D loss: 0.117695] [G loss: -0.567700]
epoch0 step820 [D loss: -0.544792] [G loss: -0.197560]
epoch0 step825 [D loss: -1.482543] [G loss: 0.438496]
epoch0 step830 [D loss: -1.472975] [G loss: 0.233960]
epoch0 step835 [D loss: -0.754758] [G loss: 0.616514]
epoch0 step840 [D loss: -0.974232] [G loss: 0.483143]
epoch0 step845 [D loss: -1.375528] [G loss: 0.529981]
epoch0 step850 [D loss: -1.162062] [G loss: 0.448648]
epoch0 step855 [D

...

epoch3 step31545 [D loss: -0.919074] [G loss: -1.223805]
epoch3 step31550 [D loss: 0.294877] [G loss: -1.263184]
epoch3 step31555 [D loss: -0.197859] [G loss: -1.137361]
epoch3 step31560 [D loss: -0.200414] [G loss: -1.085268]
epoch3 step31565 [D loss: -0.589299] [G loss: -1.204417]
epoch3 step31570 [D loss: -0.579790] [G loss: -0.779572]
epoch3 step31575 [D loss: -0.823153] [G loss: -0.437781]
epoch3 step31580 [D loss: -0.467805] [G loss: -0.572472]
epoch3 step31585 [D loss: -0.309287] [G loss: -0.931052]
epoch3 step31590 [D loss: -0.976132] [G loss: -0.740940]
epoch3 step31595 [D loss: -0.568486] [G loss: -1.841153]
epoch3 step31600 [D loss: -0.907999] [G loss: -0.985915]
epoch3 step31605 [D loss: 0.182938] [G loss: -1.129802]
epoch3 step31610 [D loss: -0.642654] [G loss: -1.301825]
epoch3 step31615 [D loss: -0.437808] [G loss: -1.310751]
epoch3 step31620 [D loss: 0.020403] [G loss: -1.498952]
epoch3 step31625 [D loss: -1.241005] [G loss: -1.092308]
epoch3 step31630 [D loss: -0.04626

epoch3 step32275 [D loss: 0.451751] [G loss: -0.843752]
epoch3 step32280 [D loss: -0.606762] [G loss: -0.759693]
epoch3 step32285 [D loss: -0.501184] [G loss: -0.847124]
epoch3 step32290 [D loss: -0.503398] [G loss: -1.389933]
epoch3 step32295 [D loss: -0.741702] [G loss: -0.422095]
epoch3 step32300 [D loss: -0.373563] [G loss: -0.358303]
epoch3 step32305 [D loss: -0.420995] [G loss: -0.220580]
epoch3 step32310 [D loss: -0.925010] [G loss: -0.836206]
epoch3 step32315 [D loss: -0.885715] [G loss: -1.080563]
epoch3 step32320 [D loss: -1.534062] [G loss: -0.973993]
epoch3 step32325 [D loss: -0.216656] [G loss: -0.937316]
epoch3 step32330 [D loss: -0.674378] [G loss: -1.185601]
epoch3 step32335 [D loss: -0.938256] [G loss: -1.055259]
epoch3 step32340 [D loss: -0.232051] [G loss: -1.274567]
epoch3 step32345 [D loss: -0.866898] [G loss: -1.243907]
epoch3 step32350 [D loss: -0.505066] [G loss: -1.752558]
epoch3 step32355 [D loss: -0.544874] [G loss: -1.073236]
epoch3 step32360 [D loss: -0.228

epoch3 step33005 [D loss: -0.951676] [G loss: -0.676336]
epoch3 step33010 [D loss: -0.231128] [G loss: -0.448698]
epoch3 step33015 [D loss: -0.564796] [G loss: 0.171045]
epoch3 step33020 [D loss: -0.702736] [G loss: -0.431867]
epoch3 step33025 [D loss: 0.003162] [G loss: -1.048763]
epoch3 step33030 [D loss: -0.565961] [G loss: -0.460958]
epoch3 step33035 [D loss: -0.758636] [G loss: -0.635280]
epoch3 step33040 [D loss: -0.637401] [G loss: -1.552022]
epoch3 step33045 [D loss: -0.362459] [G loss: -1.021153]
epoch3 step33050 [D loss: -0.507353] [G loss: -1.107270]
epoch3 step33055 [D loss: -1.039531] [G loss: -0.574058]
epoch3 step33060 [D loss: -0.925336] [G loss: -1.677624]
epoch3 step33065 [D loss: -1.320792] [G loss: -1.639658]
epoch3 step33070 [D loss: -0.990194] [G loss: -0.939839]
epoch3 step33075 [D loss: -0.037762] [G loss: -1.184335]
epoch3 step33080 [D loss: -0.350229] [G loss: -1.254511]
epoch3 step33085 [D loss: -1.255641] [G loss: -2.001266]
epoch3 step33090 [D loss: -1.3081

epoch3 step33735 [D loss: -1.002012] [G loss: -1.722176]
epoch3 step33740 [D loss: -0.513385] [G loss: -1.818133]
epoch3 step33745 [D loss: -0.385435] [G loss: -1.717720]
epoch3 step33750 [D loss: 0.153626] [G loss: -1.830232]
epoch3 step33755 [D loss: -1.235582] [G loss: -1.455909]


KeyboardInterrupt: 